# Clinical Ontology Testing

NER by itself isn't very reliable. Let's cross-reference this with the NCBO clinical ontologies, which include SNOMED-CT and many others.

In [31]:
import os 

In [32]:
dir_path = os.getcwd()
f = open(os.path.join(dir_path, "../../wmt22test.txt"), "r", encoding = "utf8")
en_sent = [line.strip() for line in f.readlines()]
f.close()
f = open(os.path.join(dir_path, "../../wmt22gold.txt"), "r", encoding = "utf8")
fr_sent = [line.strip() for line in f.readlines()]
f.close()

In [33]:
#We will obtain annotations using get_wmt22_onto_annotations.py. For now, tokenise and PoS-tag both EN and FR sentences.
import stanza

stanza.download('fr', processors='tokenize, mwt, pos', package='sequoia')
nlp_fr = stanza.Pipeline('fr', processors='tokenize, mwt, pos', package='sequoia')
stanza.download('en', processors='tokenize, pos', package='mimic')
nlp_en = stanza.Pipeline('en', processors='tokenize, pos', package='mimic')

2023-07-14 22:03:23 INFO: Downloading these customized packages for language: fr (French)...
| Processor       | Package  |
------------------------------
| tokenize        | sequoia  |
| mwt             | sequoia  |
| pos             | sequoia  |
| forward_charlm  | newswiki |
| backward_charlm | newswiki |
| pretrain        | conll17  |

2023-07-14 22:03:23 INFO: File exists: C:\Users\ethan\stanza_resources\fr\tokenize\sequoia.pt
2023-07-14 22:03:23 INFO: File exists: C:\Users\ethan\stanza_resources\fr\mwt\sequoia.pt
2023-07-14 22:03:23 INFO: File exists: C:\Users\ethan\stanza_resources\fr\pos\sequoia.pt
2023-07-14 22:03:23 INFO: File exists: C:\Users\ethan\stanza_resources\fr\forward_charlm\newswiki.pt
2023-07-14 22:03:23 INFO: File exists: C:\Users\ethan\stanza_resources\fr\backward_charlm\newswiki.pt
2023-07-14 22:03:24 INFO: File exists: C:\Users\ethan\stanza_resources\fr\pretrain\conll17.pt
2023-07-14 22:03:24 INFO: Finished downloading models and saved to C:\Users\ethan\stanza_

2023-07-14 22:03:24 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | sequoia |
| mwt       | sequoia |
| pos       | sequoia |

2023-07-14 22:03:24 INFO: Using device: cpu
2023-07-14 22:03:24 INFO: Loading: tokenize
2023-07-14 22:03:24 INFO: Loading: mwt
2023-07-14 22:03:24 INFO: Loading: pos
2023-07-14 22:03:25 INFO: Done loading processors!


2023-07-14 22:03:25 WARNING: Can not find mwt: mimic from official model list. Ignoring it.
2023-07-14 22:03:25 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package |
-----------------------------
| tokenize        | mimic   |
| pos             | mimic   |
| pretrain        | mimic   |
| forward_charlm  | mimic   |
| backward_charlm | mimic   |

2023-07-14 22:03:25 INFO: File exists: C:\Users\ethan\stanza_resources\en\tokenize\mimic.pt
2023-07-14 22:03:25 INFO: File exists: C:\Users\ethan\stanza_resources\en\pos\mimic.pt
2023-07-14 22:03:25 INFO: File exists: C:\Users\ethan\stanza_resources\en\pretrain\mimic.pt
2023-07-14 22:03:25 INFO: File exists: C:\Users\ethan\stanza_resources\en\forward_charlm\mimic.pt
2023-07-14 22:03:26 INFO: File exists: C:\Users\ethan\stanza_resources\en\backward_charlm\mimic.pt
2023-07-14 22:03:26 INFO: Finished downloading models and saved to C:\Users\ethan\stanza_resources.
2023-07-14 22:03:26 INFO: Checking 

2023-07-14 22:03:26 WARNING: Can not find mwt: mimic from official model list. Ignoring it.
2023-07-14 22:03:26 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |

2023-07-14 22:03:26 INFO: Using device: cpu
2023-07-14 22:03:26 INFO: Loading: tokenize
2023-07-14 22:03:26 INFO: Loading: pos
2023-07-14 22:03:26 INFO: Done loading processors!


In [34]:
from tqdm import tqdm
#Extract only the information we need - PoS tags and text.
fr_tagged = []
for sentence in tqdm(fr_sent):
    doc = nlp_fr(sentence)
    tokens = [{"text" : word.text, "upos" : word.upos} for sent in doc.sentences for word in sent.words]
    fr_tagged.append(tokens)

100%|██████████| 588/588 [04:24<00:00,  2.22it/s]


In [35]:
en_tagged = []
for sentence in tqdm(en_sent):
    doc = nlp_en(sentence)
    tokens = [{"text" : word.text, "upos" : word.upos} for sent in doc.sentences for word in sent.words]
    en_tagged.append(tokens)

100%|██████████| 588/588 [03:43<00:00,  2.63it/s]


In [36]:
#Word alignment
from simalign import SentenceAligner
aligner = SentenceAligner(matching_methods="a") #Argmax only; the simAlign paper stated that this gives the best performance for English-French alignment
alignments_list = []
for i in tqdm(range(len(en_tagged))):
    src_sent = [token["text"] for token in en_tagged[i]]
    tgt_sent = [token["text"] for token in fr_tagged[i]] 
    alignments = aligner.get_word_aligns(src_sent, tgt_sent)
    alignments_list.append(alignments["inter"])

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-07-14 22:51:35,500 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased
100%|██████████| 5

In [44]:
#Read in our annotations, and word-tokenise any multi-word annotations with the same EN tokeniser to ensure consistency
f = open("wmt22_onto_annotations.txt", "r", encoding = "utf8")
output = open("wmt22_onto_word_tok.txt", "w", encoding = "utf8")
for line in tqdm(f.readlines()):
    word_list = line.strip().split("\t")
    doc = nlp_en(word_list[1])
    words = [word.text for sent in doc.sentences for word in sent.words]
    for word in words:
        output.write(word_list[0] + "\t" + word + "\n")
f.close()
output.close()

100%|██████████| 6863/6863 [05:59<00:00, 19.11it/s]


In [48]:
import pandas as pd

anno_list = pd.read_csv("wmt22_onto_word_tok.txt", sep = "\t", header = None, names = ["sent_ID", "annotation"])
anno_list = anno_list.drop_duplicates().reset_index(drop=True)
anno_list["anno_lower"] = anno_list["annotation"].apply(str.lower)
anno_list = anno_list.drop(columns = "annotation")

In [56]:
#Now that we have our word-tokenised annotation list, we will query our alignments list.
import string

prospective_ne_fr = []
for i in range(len(alignments_list)):
    ne_fr_per_sentence = []
    #Filter annotations dataframe
    sentence_annos = anno_list[anno_list["sent_ID"] == i]
    for alignment in alignments_list[i]:
        src_word = en_tagged[i][alignment[0]]
        tgt_word = fr_tagged[i][alignment[1]]
        #If there is a match for our source word
        if not (sentence_annos[sentence_annos["anno_lower"] == src_word["text"].lower()].empty):
            if ((src_word["upos"] in ["PROPN", "NOUN", "ADJ", "VERB"]) & #Determiners, adverbs, prepositions, etc. are not terminologies of interest, because we are doing word-level checking
            (tgt_word["upos"] == src_word["upos"]) & #Ensure both pipelines agree on PoS tags
            (src_word["text"] not in string.punctuation) &
            (tgt_word["text"] not in string.punctuation)): #In case of incorrect tagging wrt punctuation
                ne_fr_per_sentence.append(tgt_word["text"])
    prospective_ne_fr.append(ne_fr_per_sentence)

In [58]:
sent_IDs = []
terms = []
for i in range(len(prospective_ne_fr)):
    for term in prospective_ne_fr[i]:
        sent_IDs.append(i)
        terms.append(term)
term_list = pd.DataFrame(data = {"sent_ID" : sent_IDs, "term" : terms})

In [59]:
def find_count_in_sentence_exact_match(row):
    query = row["term"]
    return len([found for found in fr_tagged[row["sent_ID"]] if query == found["text"]])

In [60]:
term_list["count"] = term_list.apply(find_count_in_sentence_exact_match, axis=1)

In [61]:
term_list = term_list.drop_duplicates().reset_index(drop=True)

In [65]:
term_list.to_csv("wmt22_onto_terms_counts.txt", sep = "\t", header = False, index = False) 

In [63]:
#Next, we will generate a separate list without casing, and aggregate counts as usual. Interestingly, we have the same number of rows, indicating that there are no terminologies which 
#appear multiple times within a single sentence, but with different casing. This means that we can stop here for now.
#term_list_uncased = term_list[["sent_ID", "term"]]
#term_list_uncased["uncased_term"] = term_list_uncased["term"].apply(str.lower)
#term_list_uncased = term_list_uncased.drop(columns = "term")
#term_list_uncased = term_list_uncased.drop_duplicates().reset_index(drop=True)